여러 종류의 classifiers 의 사용법을 익히기 위하여 앞서 다뤘던 영화 평점 분류 데이터를 이용합니다. Support Vector Machine 계열들은 데이터를 그대로 외우기 때문에 데이터의 크기가 커지면 모델의 크기도 커집니다. 그렇기 때문에 이번 실습에서는 데이터의 개수가 10k 인 작은 데이터를 이용합니다. 긍정 (label = 1), 부정 (label = -1) 의 리뷰 5k 씩으로 구성되어 있는 balanced dataset 입니다.

In [1]:
import numpy as np
import sklearn
import warnings
from lovit_textmining_dataset.navermovie_comments import load_sentiment_dataset

warnings.filterwarnings('ignore')
print(sklearn.__version__)

texts, x, y, idx_to_vocab = load_sentiment_dataset(data_name='10k', tokenize='komoran')
print(x.shape)
print(y.shape)

0.21.3
(10000, 4808)
(10000,)


## normalize

학습용 데이터 x는 row normalize를 미리 해두겠습니다. 기본은 norm='l2' 입니다. 벡터를 확률의 형식으로 (합이 1 이 되도록) 만들고 싶다면 norm 을 'l1' 으로 변경하면 됩니다.

```
x_norm_l1 = normalize(x, norm='l1')
```

In [2]:
from sklearn.preprocessing import normalize

x_norm = normalize(x, norm='l2')

## Logistic Regression

모델의 사용법을 알기 위함이니 학습과 검증 데이터를 따로 분류하지는 않겠습니다.

In [3]:
%time
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression()
logistic_regression.fit(x_norm, y)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.78 µs


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

테스트는 여러 classifier 를 이용하여 반복할테니 함수로 만들어 둡니다. 학습 정확도를 기준으로 모델의 학습력을 확인합니다.

In [4]:
def test(classifier, x, y):
    y_pred = classifier.predict(x)
    accuracy = (y == y_pred).mean()
    print(f'training accuracy = {accuracy:.4}')

test_samples = np.asarray(list(range(0,10)) + list(range(5000, 5010)))
test(logistic_regression, x, y)

training accuracy = 0.8745


Logistic Regression의 coefficient를 살펴보는 것은 이전에 다뤘습니다. 긍정, 부정을 의미하는 단어들을 살펴볼 수 있습니다.

In [5]:
# Coefficients를 array로 만듭니다. 
coefficients = logistic_regression.coef_.reshape(-1)

# Coefficient를 (index, coef)로 만든 뒤, coefficient 의 크기로 내림차순 정렬합니다. 
sorted_coefficients = sorted(enumerate(coefficients), key=lambda x:-x[1])

# 긍정적인 리뷰에 자주 등장하는 단어들을 다시 살펴봅니다.
for j, coef in sorted_coefficients[:10]:
    print('{} : {:.3}'.format(idx_to_vocab[j], coef))

관람객/NNG : 7.07
놀라/VV : 4.77
최고/NNG : 4.43
10/SN : 3.81
인생/NNP : 3.46
필요/NNG : 3.3
롭/XSA : 3.04
재미있/VA : 3.02
우주/NNG : 2.96
인터스텔라/NNP : 2.87


## Multi Layer Perceptron Classifier

Feedforward neural network 의 sklearn 의 이름이며, NN 은 parameter 를 해석하기가 조금 복잡합니다. 하지만 학습 방법은 Logistic Regression 과 동일합니다. 

Neural network 의 경우, hidden laber 의 층 수와, 각 층의 노드 수를 정해줄 수 있습니다. 아래 예시처럼 hidden layer 를 정하면, 2 개의 layers 에 각각 20, 5 개의 nodes 를 이용한다는 의미입니다. 

In [6]:
%%time
from sklearn.neural_network import MLPClassifier

neural_network = MLPClassifier(hidden_layer_sizes=(20,5))
neural_network.fit(x_norm, y)

test(neural_network, x, y)

training accuracy = 0.9795
CPU times: user 1min 19s, sys: 3min 32s, total: 4min 52s
Wall time: 24.4 s


Coefficients 는 coefs_ 에 list 형식으로 저장됩니다. 

In [7]:
print(type(neural_network.coefs_))
print(len(neural_network.coefs_))

<class 'list'>
3


In [8]:
for i, coef_ in enumerate(neural_network.coefs_):
    print('layer {} : {}'.format(i+1, coef_.shape))

layer 1 : (4808, 20)
layer 2 : (20, 5)
layer 3 : (5, 1)


Intercept 는 $h_{t-1} \times \beta_t + intercept_t$ 처럼 이전 layer 의 output 값과 $\beta$ 의 내적에 더해지는 값입니다. Logistic regression 에서의 평행이동과 같습니다.

In [9]:
neural_network.intercepts_

[array([ 0.09337455,  0.10501489,  0.05272212,  0.06463424,  0.05581306,
         0.34889031,  0.06916448,  0.30141196, -0.18363755,  0.05200041,
         0.1156127 ,  0.14034019,  0.11764886,  0.33767206,  0.05829439,
         0.09752739,  0.1578528 ,  0.08662141,  0.06460719,  0.09404335]),
 array([0.38472889, 0.32938485, 0.10315762, 0.53241528, 0.24371127]),
 array([0.42249033])]

## Support Vector Machine (Classifier)

SVM은 kernel을 정할 수 있습니다. default는 linear입니다. 

Linear kernel이 속도도 훨씬 빠릅니다. 

In [10]:
%%time
from sklearn.svm import SVC

rbf_SVM = SVC(C=1.0, kernel='rbf')
rbf_SVM.fit(x_norm, y)
test(rbf_SVM, x, y)

training accuracy = 0.774
CPU times: user 25.6 s, sys: 84 ms, total: 25.7 s
Wall time: 25.7 s


In [11]:
type(rbf_SVM.support_vectors_)

scipy.sparse.csr.csr_matrix

이런 경우는 저도 자주 보지 못합니다만, 데이터를 모두 외워버렸습니다. k-nearest neighbor 와 다를 게 없습니다.

In [12]:
rbf_SVM.support_vectors_.shape

(10000, 4808)

In [13]:
%%time

linear_SVM = SVC(C=1.0, kernel='linear')
linear_SVM.fit(x_norm, y)
test(linear_SVM, x, y)

training accuracy = 0.9088
CPU times: user 10.3 s, sys: 32 ms, total: 10.3 s
Wall time: 10.4 s


SVM 은 support vector를 직접 확인할 수가 있습니다. 

linear SVM 의 support vector들을 확인해 보겠습니다. (4167, 4808) 은 4,808 차원 (단어 개수) 의 4,167 개의 support vectors 가 있다는 의미입니다. 이 숫자도 매우 큽니다. 이는 features 가 반복되지 않는 (각 리뷰별로 다양한 단어들이 이용되는) 데이터셋이기 때문입니다.

In [14]:
linear_SVM.support_vectors_.shape

(4167, 4808)

원래 데이터 X 에서의 각 support vectors 의 index 가 support_ 에 저장되어 있습니다.

In [15]:
linear_SVM.support_

array([   0,    1,    2, ..., 9989, 9994, 9996], dtype=int32)

SVC.dual\_coef\_에는 alpha가 학습되어 있습니다. 이 값이 negative이면 negative class 입니다. 

In [16]:
linear_SVM.dual_coef_

<1x4167 sparse matrix of type '<class 'numpy.float64'>'
	with 4167 stored elements in Compressed Sparse Row format>

In [17]:
import numpy as np

hist, bin_edges = np.histogram(
    linear_SVM.dual_coef_.data,
    bins=20
)

대부분의 support vectors 의 weight $\alpha$ 의 값이 -1 아니면 1 에 가깝습니다. 사실상 대부분의 리뷰들이 비슷한 중요도를 지닙니다.

In [18]:
for b, e, hist_i in zip(bin_edges, bin_edges[1:], hist):
    print('({:.3} ~ {:.3}) : {}'.format(b, e, hist_i))

(-1.0 ~ -0.9) : 1637
(-0.9 ~ -0.8) : 56
(-0.8 ~ -0.7) : 45
(-0.7 ~ -0.6) : 61
(-0.6 ~ -0.5) : 43
(-0.5 ~ -0.4) : 64
(-0.4 ~ -0.3) : 59
(-0.3 ~ -0.2) : 52
(-0.2 ~ -0.1) : 56
(-0.1 ~ 0.0) : 54
(0.0 ~ 0.1) : 33
(0.1 ~ 0.2) : 34
(0.2 ~ 0.3) : 33
(0.3 ~ 0.4) : 47
(0.4 ~ 0.5) : 45
(0.5 ~ 0.6) : 35
(0.6 ~ 0.7) : 31
(0.7 ~ 0.8) : 39
(0.8 ~ 0.9) : 48
(0.9 ~ 1.0) : 1695


## Naive Bayes

naive bayes 역시 학습 속도가 매우 빠릅니다. 

In [19]:
%%time

from sklearn.naive_bayes import BernoulliNB

bernoulli_naive_bayes = BernoulliNB()
bernoulli_naive_bayes.fit(x_norm, y)
test(bernoulli_naive_bayes, x, y)

training accuracy = 0.8922
CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 19.7 ms


Naive Bayes 역시 binary classification 에서는 (1, n_terms) 의 coefficient 가 저장되어 있습니다. 숫자가 클수록 긍정 리뷰에 발생하는 단어라는 의미입니다.

일반적으로 확률의 곲을 이용하는 모델들은 계산의 편의성을 위하여 log 확률을 이용합니다. 확률의 누적곲에 log 를 취하면 덧샘이 되기 때문입니다. Naive Bayes 의 coefficient 의 값은 log 가 취해진 확률값입니다.

In [20]:
bernoulli_naive_bayes.coef_

array([[-6.57168296, -8.51759311, -4.32793837, ..., -5.95264375,
        -7.13129875, -5.33953928]])

하지만 Naive Bayes 는 단어 빈도수 정보까지 coefficient 에 저장합니다. Classsification 에 불필요한 단어가 많이 포함되어 있다면 이들에 의해 classification 의 성능이 저하될 수도 있습니다. 즉 Naive Bayes 를 잘 이용하기 위해서는 정보력이 적은 (frequent) 단어들은 제거해 주면 도움이 됩니다.

그리고 판별에 명확한 힌트가 되는 단어들로만 구성된 데이터셋이라면 coefficient 의 해석도 자연스럽습니다.

In [21]:
# Coefficients를 array로 만듭니다. 
coefficients = bernoulli_naive_bayes.coef_.reshape(-1)

# Coefficient를 (index, coef)로 만든 뒤, coefficient 의 크기로 내림차순 정렬합니다. 
sorted_coefficients = sorted(enumerate(coefficients), key=lambda x:-x[1])

for j, coef in sorted_coefficients[:10]:
    print('{} : {:.3}'.format(idx_to_vocab[j], coef))

관람객/NNG : -0.901
ㄴ/ETM : -1.07
보/VV : -1.13
영화/NNG : -1.16
이/VCP : -1.3
다/EC : -1.35
하/XSV : -1.44
는/ETM : -1.45
고/EC : -1.51
이/JKS : -1.52


## Decision Tree

In [22]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(
    max_depth=50,
    min_samples_split=2,
    min_samples_leaf=5,
    max_features=None,
    max_leaf_nodes=None,
    class_weight=None
)

decision_tree.fit(x_norm, y)
test(decision_tree, x, y)

training accuracy = 0.8103


In [23]:
decision_tree = DecisionTreeClassifier(
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=5,
    max_features=None,
    max_leaf_nodes=None,
    class_weight=None
)

decision_tree.fit(x_norm, y)
test(decision_tree, x, y)

training accuracy = 0.787


In [24]:
import numpy as np
np.where(decision_tree.feature_importances_ > 0)[0]

array([   2,  161,  162,  178,  179,  184,  206,  257,  336,  370,  480,
        494,  559,  585,  842,  877,  882,  931,  985, 1006, 1018, 1031,
       1032, 1074, 1111, 1152, 1255, 1299, 1406, 1557, 1591, 1662, 2078,
       2114, 2512, 2601, 2716, 2734, 2741, 2808, 2851, 2983, 2996, 3098,
       3359, 3368, 3408, 3435, 3551, 3697, 3706, 3876, 3947, 3973, 3998,
       4000, 4008, 4176, 4213, 4456, 4511, 4517, 4518, 4558])

In [25]:
decision_tree.feature_importances_.shape

(4808,)

몇 개의 단어만을 이용하여 분류를 하다보니 분류가 제대로 이뤄지지 않기도 합니다. 게다가 분류에 유용한 features 가 아닌 경우도 많습니다. 또한 `크리스토퍼 놀란` 때문에 `놀라/VV`, `놀라/NNP` 와 같은 잘못된 features 들도 생성되었음을 볼 수 있습니다.

In [27]:
[(idx_to_vocab[idx], importancy) for idx, importancy in 
 sorted(
     filter(lambda x:x[1]>0,
            enumerate(decision_tree.feature_importances_)),
     key=lambda x:-x[1]
 )
][:10]

[('관람객/NNG', 0.451064896067875),
 ('놀라/VV', 0.10633635178663352),
 ('최고/NNG', 0.08842323250056373),
 ('번/NNB', 0.038251594698815006),
 ('경이/NNG', 0.034499105314828776),
 ('필요/NNG', 0.029883137110439176),
 ('수/NNB', 0.028917042393979457),
 ('놀라/NNP', 0.027173280668945796),
 ('아깝/VA', 0.023679729407711188),
 ('인터스텔라/NNP', 0.020905808382841992)]

## Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(
    n_estimators=100,
    min_samples_leaf=10,
    max_depth=4,
    max_features=16 # or ['sqrt', 'log2', None] if None then x_norm.shape[1]
)

random_forest.fit(x_norm, y)
test(random_forest, x, y)

training accuracy = 0.7816


estimators 에 각각의 decision tree 가 저장되어 있습니다.

In [29]:
len(random_forest.estimators_)

100

이를 확인해 볼 수 있습니다.

In [30]:
random_forest.estimators_[0]

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
                       max_features=16, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=332332089, splitter='best')

각각의 decision tree 역시 classifier 로 이용 가능합니다.

In [31]:
test(random_forest.estimators_[0], x, y)

training accuracy = 0.4981


100 개씩 decision tree 를 추가할 수도 있습니다.

In [32]:
# incrementally add new random classifiers
for _ in range(5):
    random_forest.set_params(
        n_estimators = random_forest.n_estimators + 100,
        warm_start = True
    )
    random_forest.fit(x_norm, y)
    print(len(random_forest.estimators_))

200
300
400
500
600


predict_proba 함수를 이용하면 각 instance 가 클래스에 속할 확률을 return 합니다. 아래와 같은 결과는 거의 대부분의 확률이 0.5 에 가깝습니다. 이는 데이터를 분류하는 경계가 뚜렷하지 않다는 의미이기도 합니다. 앙상블 모델을 구성하는 base model 들이 weak classifiers 이기 때문입니다.

In [33]:
random_forest.predict_proba(x_norm[:10])

array([[0.5032677 , 0.4967323 ],
       [0.50828244, 0.49171756],
       [0.50236723, 0.49763277],
       [0.50641861, 0.49358139],
       [0.49371089, 0.50628911],
       [0.52492934, 0.47507066],
       [0.50942032, 0.49057968],
       [0.50474634, 0.49525366],
       [0.50831127, 0.49168873],
       [0.50244556, 0.49755444]])

실제로 모든 학습 데이터에 대하여 예측 확률의 평균을 계산하면 0.51 을 조금 넘습니다.

In [34]:
random_forest.predict_proba(x_norm).max(axis=1).mean()

0.5122370033309187